In [1]:
import spacy
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from collections import Counter

# nlp = spacy.load('en_core_web_sm')
nlpmd = spacy.load('en_core_web_md')
nlplg = spacy.load('en_core_web_lg')


In [2]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":  msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [3]:
data.head()

,class_to_exec,content,date,from,subject,to
0,Booking,"Dear Rohit,\n \nAs per our conversation a whil...",None,"[(, Meemendra), (, ), (, meemendra.kumar@andbe...",Mr. Nagdev X 04,"[(, Gateway)]"
1,Booking,"Hi Agnes,\n \nI have not received a response t...",None,"[(, Kurien), (, Vinay), (, ), (, vkurien@jones...",RE: Rate Proposal - Taj Connemara,"[(Reservations, Reservations@tajhotels.com)]"
2,Booking,"As per corporate holiday plan, kindly arrange ...",None,"[(, tehp@tatasteel.com[mailto:tehp@tatasteel.c...",TEHP/18/12809: Booking request of Arun Gadamsh...,"[(Reservations Coorg, reservations.coorg@tajho..."
3,Booking,"Dear Gampa ji,\n \nGreetings of the day !\n \n...",None,"[(, INDIAN), (, ), (, indianjourney@gmail.com)]",Valley Period/3 double room/Lerner group,"[(, Neeraj)]"
4,Cancellation,Hi\n\nPlease cancel my booking .,None,"[(, rajivbhatia@ase.ae[mailto:rajivbhatia@ase....",Re: Taj Chandigarh Payment link for reservatio...,"[(Reservations, Reservations@tajhotels.com)]"


In [78]:
body = ["Serial number"]

entityData = []
for cnt in body:
    entityCh = []
    contentSentence = nlplg(cnt)
    for sent in contentSentence.sents:
        entity = nlplg(str(sent))
        for token in entity.ents:
            entityData.append({"text":sent, "Entity":token.text, "label":token.label_, "start_char":token.start_char, "end_char":token.end_char})
    #entityData.append({"body":cnt, "entities":entityCh})

In [79]:
entities = pd.DataFrame(entityData)

In [80]:
# print(entities["body"][0])

In [81]:
entities["Target"] = True

In [83]:
entities

,Target


In [84]:
entities

,Target


In [65]:
entities["Target"] = entities["Entity"].apply(lambda x: True if x.strip() != "" else False)

In [69]:
entities.to_csv("../datawe/raw/Email_Classification/entityTraining.csv")